In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, Bidirectional, BatchNormalization, Conv1D, MaxPooling1D, Reshape
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
np.random.seed(0)

In [3]:
data=pd.read_csv("multi_data.csv")
data

,Unnamed: 0,count,logged_in,srv_serror_rate,serror_rate,dst_host_serror_rate,dst_host_same_srv_rate,dst_host_srv_serror_rate,dst_host_srv_count,same_srv_rate,...,flag_S3,flag_SF,flag_SH,intrusion,Dos,Probe,R2L,U2R,normal,label
0,0,-0.717045,-0.809262,-0.631929,-0.637209,-0.639532,-0.782367,-0.624871,-0.818890,0.771283,...,0,1,0,4,0,0,0,0,1,normal
1,1,-0.620982,-0.809262,-0.631929,-0.637209,-0.639532,-1.161030,-0.624871,-1.035688,-1.321428,...,0,1,0,4,0,0,0,0,1,normal
2,2,0.339648,-0.809262,1.605104,1.602664,1.608759,-0.938287,1.618955,-0.809857,-1.389669,...,0,0,0,0,1,0,0,0,0,Dos
3,3,-0.690846,1.235694,-0.184522,-0.189235,-0.572083,1.066401,-0.602433,1.258754,0.771283,...,0,1,0,4,0,0,0,0,1,normal
4,4,-0.472521,1.235694,-0.631929,-0.637209,-0.639532,1.066401,-0.624871,1.258754,0.771283,...,0,1,0,4,0,0,0,0,1,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,125968,0.872361,-0.809262,1.605104,1.602664,1.608759,-0.938287,1.618955,-0.818890,-1.184947,...,0,0,0,0,1,0,0,0,0,Dos
125969,125969,-0.717045,-0.809262,-0.631929,-0.637209,-0.639532,0.977304,-0.624871,1.159389,0.771283,...,0,1,0,4,0,0,0,0,1,normal
125970,125970,-0.725778,1.235694,-0.631929,-0.637209,0.979238,-0.893738,-0.624871,-0.773724,0.771283,...,0,1,0,4,0,0,0,0,1,normal
125971,125971,0.523041,-0.809262,1.605104,1.602664,1.608759,-1.094207,1.618955,-0.972455,-1.366922,...,0,0,0,0,1,0,0,0,0,Dos


In [4]:
data=data.drop("Unnamed: 0",axis=1)
X=data.iloc[:,0:92]
Y=data[['Dos','normal','Probe','R2L','U2R']]

In [5]:
X_tr,X_te,y_tr,y_te=train_test_split(X,Y,test_size=0.25,random_state=42)

In [6]:
model = Sequential()
model.add(Conv1D(64, kernel_size=92, padding="same",activation="relu",input_shape=(92,1)))
model.add(MaxPooling1D(pool_size=(5)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(64, return_sequences=False))) 
model.add(Reshape((128, 1), input_shape = (128, )))
model.add(MaxPooling1D(pool_size=(5)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(128, return_sequences=False))) 
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [7]:
history=model.fit(X_tr,y_tr,epochs=15,batch_size=32,validation_split=0.3)

Epoch 1/15
2067/2067 [==============================] - 51s 22ms/step - loss: 0.1451 - accuracy: 0.9567 - val_loss: 0.0963 - val_accuracy: 0.9706
Epoch 2/15
2067/2067 [==============================] - 44s 21ms/step - loss: 0.0958 - accuracy: 0.9698 - val_loss: 0.0863 - val_accuracy: 0.9691
Epoch 3/15
2067/2067 [==============================] - 44s 21ms/step - loss: 0.0874 - accuracy: 0.9716 - val_loss: 0.0995 - val_accuracy: 0.9716
Epoch 4/15
2067/2067 [==============================] - 44s 22ms/step - loss: 0.0800 - accuracy: 0.9741 - val_loss: 0.0709 - val_accuracy: 0.9774
Epoch 5/15
2067/2067 [==============================] - 45s 22ms/step - loss: 0.0768 - accuracy: 0.9757 - val_loss: 0.0732 - val_accuracy: 0.9762
Epoch 6/15
2067/2067 [==============================] - 47s 23ms/step - loss: 0.0732 - accuracy: 0.9762 - val_loss: 0.0672 - val_accuracy: 0.9769
Epoch 7/15
2067/2067 [==============================] - 45s 22ms/step - loss: 0.0719 - accuracy: 0.9766 - val_loss: 0.0838 -

In [8]:
model.save("paper_3_BiLSTM.h5")

In [9]:
y_pr=model.predict(X_te)
y_pr=np.argmax(y_pr, axis=1)
y_te=y_te.to_numpy()
y_te=np.argmax(y_te,axis=1)

In [10]:
print(accuracy_score(y_te,y_pr))

0.9772020067314409


In [11]:
print(confusion_matrix(y_te,y_pr))

[[11256   218    10     0     0]
 [   22 16551   161    40     0]
 [  115    85  2747     0     0]
 [    0    52     0   222     0]
 [    0    11     0     4     0]]


In [12]:
print(classification_report(y_te,y_pr,target_names=['Dos','normal','Probe','R2L','U2R']))

              precision    recall  f1-score   support

         Dos       0.99      0.98      0.98     11484
      normal       0.98      0.99      0.98     16774
       Probe       0.94      0.93      0.94      2947
         R2L       0.83      0.81      0.82       274
         U2R       0.00      0.00      0.00        15

    accuracy                           0.98     31494
   macro avg       0.75      0.74      0.75     31494
weighted avg       0.98      0.98      0.98     31494



C:\Users\getch\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\getch\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\getch\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo